# Imports

In [1]:
# <your imports>
import torch
from torch.optim import Adam
import torch.nn as nn
from tqdm import tqdm
from omegaconf import OmegaConf
from preprocessing import Preprocessing
from trainer import Trainer

# models
from models import BertClassifier

# Data loading/preprocessing

In [2]:
preprocessor = Preprocessing()

result = preprocessor.get_dataloaders()

train_dl = result["train_dl"]
val_dl = result["val_dl"]

# Model initialization

In [4]:
trainer = Trainer()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Initialized training config with params: {'dataset_name': 'ruSentNE', 'batch_size': 2, 'device': 'cpu:0', 'dataloader_shuffle': True, 'pretrained_model': 'bert-base-uncased', 'num_classes': 3, 'num_workers': 1, 'lr': 1e-06, 'epochs': 5, 'eps': 1e-08, 'optimizer': 'AdamW', 'scheduler': None}


## Train loop

In [ ]:
trainer.train(train_dl, val_dl)

# Experiments